In [1]:
import ee
import datetime
from datetime import date, timedelta
ee.Authenticate()
ee.Initialize()



Successfully saved authorization token.


In [2]:
%pip install geemap

     ---------------------------------------- 2.1/2.1 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 102.3/102.3 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 16.1 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 11.7 MB/s eta 0:00:00
     ---------------------------------------- 98.6/98.6 kB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 19.5 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 14.8 MB/s eta 0:00:00
     -------------------------------------- 246.0/246.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 130.5/130.5 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB 2.8 MB/s eta 0:00:00
  

In [2]:
import geemap
Map = geemap.Map(center = [39.027884, -84.137946], zoom = 10)
Map.add_basemap('SATELLITE') # Add satellite image basemap
Map

In [ ]:
def sentinel2Availability(ee, dt, lat, lon, delta, cloud='true'):
# Function to mask clouds using the Sentinel-2 QA band.
          # use gee map sentinel2_timeseries to find the image
    # sentinel2_timeseries(roi = None, start_year = 2015, end_year = 2019, start_date = '01-01', end_date = '12-31'):
    # imgs = sentinel2_timeseries(ee.Geometry.Point(lon,lat),date.year,date.year,'{:02d}-{:02d}'.format(date.month,date.day))
    roi = ee.Geometry.Point(lon, lat)
    if delta.days == 0:
        start = dt

        end = dt + datetime.timedelta(days = 1)
    else:
        start = dt - delta
        end = dt + delta

    date1 = ee.Date.fromYMD(start.year, start.month, start.day)
    date2 = ee.Date.fromYMD(end.year, end.month, end.day)

    # print(date1,date2)
    images = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(date1, date2) \
        .sort('CLOUDY_PIXEL_PERCENTAGE')
# imgs = sentinel2_timeseries(ee.Geometry.Point(lon,lat),date.year,date.year,'{:02d}-{:02d}'.format(date.month,date.day))

    try:
        images.getInfo()
    except:
        return 'null'
    image1 = images.first()
    try:
        image1.getInfo()
    except:
        return 'null'
    return image1

def ImageExist(img):
    try:
        if img.getInfo() == None:
            return 'false'
        else:
            return 'true'
    except:
        return 'false'

In [ ]:
def OLIAvailability(ee, dt, lat, lon, delta, cloud='true'):
# Function to mask clouds using the Sentinel-2 QA band.
          # use gee map sentinel2_timeseries to find the image
    # sentinel2_timeseries(roi=None, start_year=2015, end_year=2019, start_date='01-01', end_date='12-31'):
    # imgs = sentinel2_timeseries(ee.Geometry.Point(lon,lat),date.year,date.year,'{:02d}-{:02d}'.format(date.month,date.day))
    roi = ee.Geometry.Point(lon, lat)
    if delta.days == 0:
        start = dt

        end = dt + datetime.timedelta(days=1)
    else:
        start = dt - delta
        end = dt + delta

    date1 = ee.Date.fromYMD(start.year, start.month, start.day)
    date2 = ee.Date.fromYMD(end.year, end.month, end.day)

    # print(date1,date2)
    images = ee.ImageCollection('LANDSAT/LC09/C02/T1') \
        .filterBounds(roi) \
        .filterDate(date1, date2) \
        .sort('CLOUD_COVER	')
# imgs = sentinel2_timeseries(ee.Geometry.Point(lon,lat),date.year,date.year,'{:02d}-{:02d}'.format(date.month,date.day))

    try:
        images.getInfo()
    except:
        return 'null'
    image1 = images.first()
    try:
        image1.getInfo()
    except:
        return 'null'
    return image1

def ImageExist(img):
    try:
        if img.getInfo()==None:
            return 'false'
        else:
            return 'true'
    except:
        return 'false'

In [ ]:
# search parameters are set here
pnt_date = date(2021, 11, 4)
lat =  39.027884
lon = -84.137946
delta = timedelta(days = 10)


In [ ]:
#from sentinel_search import sentinel2Availability, ImageExist

#  search sentinel first
img = OLIAvailability(ee,pnt_date, lat, lon, delta, True)
bands = ['B1','B2','B3','B4','B5','B7']
vis = {'bands': ['B7', 'B5', 'B3'],'min': [0, 0, 0], 'max': [1000,1000, 1000]} #reflectance * 10000
if ImageExist(img) == 'true':
    ac_date = ee.Date(img.get('system:time_start')).format('MM/dd/YYYY');
    print('Found an ' + ' image for date:', pnt_date, ' at date:', ac_date.getInfo())
    img1 = img.select(['B2','B3','B4','B5','B6','B7'],['B1','B2','B3','B4','B5','B7'])
    
    #the band names are defined to match Landsat TM bands
    
    #Map.addLayer(img1, vis,'Sentinel')
else:
    print('No image was found, try another date')

Found an  image for date: 2021-11-04  at date: 11/05/2021


In [ ]:
roi = ee.Geometry.Polygon(
        [[[-84.2676, 39.0968],
          [-84.2676, 38.9562],
          [-84.0172, 38.9562],
          [-84.0172, 39.0968]]])
image_studyarea = img1.clip(roi)

In [ ]:
ndvi = image_studyarea.normalizedDifference((['B4', 'B3']))
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
Map.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': palette}, 'NDVI')

In [ ]:
task1 = ee.batch.Export.image.toDrive(image = ndvi, description = "ndvi", fileNamePrefix = "ndvi", fileFormat = "GeoTIFF", scale = 10, folder = "anyFolder")
task1.start()


In [ ]:
task1.status()

{'state': 'COMPLETED',
 'description': 'ndvi',
 'creation_timestamp_ms': 1674846333683,
 'update_timestamp_ms': 1674846404189,
 'start_timestamp_ms': 1674846370116,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/16Fv18Xa8CEPHuY6uFSU47EIwvy7DESMJ'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 7.587317943572998,
 'id': '3YKQVXLFCNVPGQQB4NO3SOUI',
 'name': 'projects/earthengine-legacy/operations/3YKQVXLFCNVPGQQB4NO3SOUI'}